# A Short Introduction to the ChatGPT API



The ChatGPT is relatively easy to use. It has object bindings in most major languages, so you can use your language of choice. 


## Getting Started


To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. Save the key to a file called "key.txt"

Install the `openai` Python package with `pip install --upgrade openai`, and run the code below to read the key.

In [ ]:
import openai

# Load and set API key
openai.api_key = open("key.txt", "r").read().strip("\n")

## Making a Request

Start writing your first request - you need to specify the model and provide the user prompt. 

The "user" role represents the user's input or message within the conversation. The prompt can include questions, commands, or any other content the user wants to communicate.

In [ ]:
user_prompt = "When was the first moon landing?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": user_prompt}],
)

print(completion)

The completion has the response content, as well as usage information. Responses are messages from the "agent" role. What we really want to extract, though, is the actual message content. 

In [ ]:
reply = completion.choices[0].message.content
print(reply)

## Using the "system" Role

The "system" role is used to provide high-level instructions or setting the behavior of the assistant. It guides the conversation or specifies the desired outcome. This information is typically provided at the beginning of the conversation.

In [ ]:
user_prompt = "When was the first moon landing?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Always reply in French."},
        {"role": "user", "content": user_prompt},
    ],
)

reply = completion.choices[0].message.content
print(reply)

ChatGPT is most useful when it has context from the conversation. You will need to maintain chat history yourself, by keeping the "system" and "user" messages as well as the "assistant" reponses in a list, and passing this list into every API call.

## Maintaining Context

ChatGPT is a REST API, and does not maintain sessions. However, the client can maintain it's own chat history, and present that on an API call for additional context on the conversation. To be effective, you would need to maintain both the user questions and the ChatGPT responses. In it's simplest form, it is a list of chat messages. 

In [ ]:
user_prompt = "What is the moon's circumference in km?"

message_history = []
message_history.append(
    {"role": "user", "content": user_prompt}
)

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=message_history
)

# Print the response:
reply_content = completion.choices[0].message.content
print(reply_content)

Let us store the response in the history as well, so we can make a context-sensitive prompt.

In [ ]:
message_history.append({"role": "assistant", "content": f"{reply_content}"})

Now, ask an ambiguous question, which is answered in context.

In [ ]:
user_prompt = "How far is it?"

message_history.append({"role": "user", "content": user_prompt})

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

reply_content = completion.choices[0].message.content
print(reply_content)

## Making Function Calls

ChatGPT does not have access to real-time data such as the weather, or proprietary information from your company that you may want to access. You can use the power of ChatGPT to understand a request from the user, and call functions that you define.

Let us say you have defined a function to get the weather. (In this case it returns hard-coded information.) The function can take parameters.

In [ ]:
import json


def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

If you describe this function and it's parameters to ChatGPT, ChatGPT will determine the intent to use this function.

In [ ]:
user_prompt = "What's the weather like in Boston?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": user_prompt}],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit for temperature",
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call="auto",
)

We name and describe the function, then describe the parameters that to pass to this function. ChatGPT relies on this description to help identify whether to call the function, and what the arguments should be. 

If we specify `function_call="auto"`, ChatGPT will try to fulfill the function parameters. If set to `none` you can force no function to be detected. 


Here is the prompt returned:

In [ ]:
reply_content = completion.choices[0]
reply_content

Notice that there is no content, but instead a function call, with extracted parameters in a JSON object. It is your responsibility to call the function if you would like to do that.

## References

- Based on https://github.com/Sentdex/ChatGPT-API-Basics
- See [ChatGPT API in Python by sentdex](https://www.youtube.com/watch?v=c-g6epk3fFE)